In [169]:
from statsmodels.tsa import holtwinters as hw
import statsmodels.api as sm
import pandas as pd
import numpy as np
import plotly.express as px
import plotly
import plotly.graph_objs as go

FREQ = 'QS'

## Данные

In [299]:

# data_all = sm.datasets.macrodata.load_pandas().data
# data_all = data_all.set_index(pd.date_range('1959Q1',
#                                     '2009Q3',
#                                      freq=FREQ
#                                    )
#                      )


# data = data_all.loc[:,['infl']].iloc[:,0]
# # data = data.drop(columns = ['YEAR']).iloc[:,0]

# fig = px.line(x=data.index, y=data.values);
# fig.update_layout(title='Inflation rate',
#                    xaxis_title='Time',
#                    yaxis_title='Number of spots');
# fig.update_xaxes(rangeslider_visible=True);
# fig.show()


# Read in data and convert index to a datetime
period = 'Monthly'
df = pd.read_csv(f'data_seminars/m4_sample/{period}_sample.csv',
                 index_col=0)

# Ряды не размечены по датам, поэтому для удобства возьмём произвольную точку старта
df.index = pd.date_range(start='01-01-2000',
                         periods=df.shape[0],
                         freq='M')


ts = df[np.random.choice(df.columns)]


fig = px.line(x=ts.index, y=ts.values);
fig.update_layout(title='Some forecasting model',
                   xaxis_title='Time',
                   yaxis_title='Value');
fig.update_xaxes(rangeslider_visible=True);
fig.show()


def forplot(mul=False):
    fig = go.Figure()

    fig.add_trace(go.Scatter( x=ts.index,
                              y=ts.values)
                 )
    
    if mul:
         fig.add_trace(go.Scatter(x=forecs.index,
                             y=forecs.values))
    else:
        fig.add_trace(go.Scatter(x=forecs.index,
                                 y=forecs['mean']))

    fig.update_xaxes(rangeslider_visible=True);
    fig.show()



## Простое экспоненциальное сглаживание, ETS(A,N,N)

In [300]:
mod1 = exp = sm.tsa.statespace.ExponentialSmoothing(ts)
res = exp.fit()
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()

                        Exponential Smoothing Results                         
Dep. Variable:                col_535   No. Observations:                   77
Model:                   ETS(A, N, N)   Log Likelihood                -552.188
Date:                Tue, 18 Aug 2020   AIC                           1110.376
Time:                        21:20:11   BIC                           1117.407
Sample:                    01-31-2000   HQIC                          1113.188
                         - 05-31-2006   Scale                        99177.205
Covariance Type:                  opg                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
smoothing_level     0.0895      0.063      1.416      0.157      -0.034       0.213
initial_level    1042.0745    178.799      5.828      0.000     691.635    1392.514
Ljung-Box (Q):                  

## Экспоненциальное сглаживание с сезонностью, ETS(A,N,A)

In [301]:
mod1 = exp = sm.tsa.statespace.ExponentialSmoothing(ts, seasonal=12)
res = exp.fit()
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()


                        Exponential Smoothing Results                         
Dep. Variable:                col_535   No. Observations:                   77
Model:                   ETS(A, N, A)   Log Likelihood                -462.669
Date:                Tue, 18 Aug 2020   AIC                            955.337
Time:                        21:20:16   BIC                            990.494
Sample:                    01-31-2000   HQIC                           969.400
                         - 05-31-2006   Scale                         9696.183
Covariance Type:                  opg                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
smoothing_level         0.0001      0.019      0.005      0.996      -0.037       0.037
smoothing_seasonal      0.9999      0.210      4.758      0.000       0.588       1.412
initial_level   

## Линейная модель Хольта-Винтерса, ETS(A,A,N)

In [302]:
mod1 = exp = sm.tsa.statespace.ExponentialSmoothing(ts,
                                                    trend=True)
res = exp.fit()
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()


                        Exponential Smoothing Results                         
Dep. Variable:                col_535   No. Observations:                   77
Model:                   ETS(A, A, N)   Log Likelihood                -547.501
Date:                Tue, 18 Aug 2020   AIC                           1105.003
Time:                        21:20:22   BIC                           1116.722
Sample:                    01-31-2000   HQIC                          1109.690
                         - 05-31-2006   Scale                        87810.782
Covariance Type:                  opg                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
smoothing_level     0.0001      0.160      0.001      0.999      -0.313       0.313
smoothing_trend     0.0001      0.028      0.004      0.997      -0.055       0.056
initial_level     978.3817     8

## Линейная модель Хольта-Винтерса с сезонностью, ETS(A,A,A)

In [303]:
mod1 = exp = sm.tsa.statespace.ExponentialSmoothing(ts,
                                                    trend=True,
                                                    damped_trend=True,
                                                    seasonal=12)
res = exp.fit()
print(res.summary())


forecs = res.get_forecast(50).summary_frame()

forplot()





                        Exponential Smoothing Results                         
Dep. Variable:                col_535   No. Observations:                   77
Model:                  ETS(A, Ad, A)   Log Likelihood                -440.493
Date:                Tue, 18 Aug 2020   AIC                            916.986
Time:                        21:20:27   BIC                            959.174
Sample:                    01-31-2000   HQIC                           933.861
                         - 05-31-2006   Scale                         5450.668
Covariance Type:                  opg                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
smoothing_level         0.0001      0.037      0.003      0.998      -0.073       0.074
smoothing_trend         0.0001      0.004      0.025      0.980      -0.008       0.008
smoothing_season

## Альтернативная реализация с мультипликативными трендом и сезонностью

In [304]:
mod1 = exp = sm.tsa.ExponentialSmoothing(ts,
                                        trend='mul',
                                        damped=True,
                                        seasonal='mul',
                                        seasonal_periods=12)
res = exp.fit()
print(res.summary())


forecs = res.forecast(50)

forplot(mul=True)


# hw.ExponentialSmoothing()


C:\Users\The_sun\Anaconda3\envs\ts_course\lib\site-packages\statsmodels\tsa\holtwinters.py:725: RuntimeWarning:

invalid value encountered in less_equal

C:\Users\The_sun\Anaconda3\envs\ts_course\lib\site-packages\statsmodels\tsa\holtwinters.py:731: RuntimeWarning:

invalid value encountered in greater_equal



                       ExponentialSmoothing Model Results                       
Dep. Variable:                    endog   No. Observations:                   77
Model:             ExponentialSmoothing   SSE                         140818.799
Optimized:                         True   AIC                            612.380
Trend:                   Multiplicative   BIC                            652.224
Seasonal:                Multiplicative   AICC                           625.713
Seasonal Periods:                    12   Date:                 Tue, 18 Aug 2020
Box-Cox:                          False   Time:                         21:20:46
Box-Cox Coeff.:                    None                                         
                          coeff                 code              optimized      
---------------------------------------------------------------------------------
smoothing_level               0.1832129                alpha                 True
smoothing_slope          